### Extracting The Datasets

In [ ]:
import io
import time
from datetime import timedelta
import gensim

if __name__ == '__main__':
    start_time = time.time()
    print('Streaming wiki...')
    id_wiki = gensim.corpora.WikiCorpus(
        './datasets/2023/bz2/idwiki-20230520-pages-articles-multistream.xml.bz2',
        dictionary={}, lower=True
    )
    
    article_count = 0
    with io.open('./datasets/2023/txt/idwiki_20230520.txt', 'w', encoding='utf-8') as wiki_txt:
        for text in id_wiki.get_texts():

            wiki_txt.write(" ".join(text) + '\n')
            article_count += 1

            if article_count % 10000 == 0:
                print('{} articles processed'.format(article_count))
        print('total: {} articles'.format(article_count))

    finish_time = time.time()
    print('Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))

### Training Word2Vec

In [ ]:
import time
import multiprocessing
from datetime import timedelta
from gensim.models import word2vec

if __name__ == '__main__':
    start_time = time.time()
    print('Training Word2Vec Model...')
    sentences = word2vec.LineSentence('./datasets/2023/txt/idwiki_new_lower_latest.txt')
    id_w2v = word2vec.Word2Vec(sentences, vector_size=200, workers=multiprocessing.cpu_count()-1, sg=1, window=2)
    id_w2v.save('model/windows_2/idwiki_word2vec_200_skip-gram_window_2_new_lower.model')
    finish_time = time.time()

    print('Finished. Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))
    

In [10]:
import time
import multiprocessing
from datetime import timedelta
from gensim.models import word2vec

start_time = time.time()
print('Training Word2Vec Model...')
# Paths to your two datasets

dataset1_path = './datasets/2023/txt/idwiki_20230520.txt'
dataset2_path = './datasets/2023/txt/idwiki_new_lower_latest.txt'

def combine_datasets(dataset1_path, dataset2_path):
    for sentence in word2vec.LineSentence(dataset1_path):
        yield sentence
    for sentence in word2vec.LineSentence(dataset2_path):
        yield sentence

# Load and combine the datasets using the custom generator
sentences = list(combine_datasets(dataset1_path, dataset2_path))  # Convert to a list

id_w2v = word2vec.Word2Vec(sentences, vector_size=200, workers=multiprocessing.cpu_count() - 1, sg=1, window=2)

# Save the trained model
id_w2v.save('./model/model_new/windows_2_new_multiple_corpus/idwiki_word2vec_200_skip-gram_window_2_new_lower.model')
finish_time = time.time()
print('Finished. Elapsed time: {}'.format(timedelta(seconds=finish_time - start_time)))

Training Word2Vec Model...
